In [1]:
import json
# import requests
import pandas as pd
import numpy as np
import os
os.chdir('../..')
# URL = "https://open-innovations.github.io/council-website-emissions/data/index.json"

Work out which places are active and which are inactive. Assume anything with data in the 'Value' column for the most recent year's data is active, otherwise it is no longer used.

In [2]:
data = pd.read_csv('data/vacant-homes/AllCombined_Cleaned_2024.csv')
latest_year = data.Year.max()

dwellings_slice = data.loc[(data['Measure']=='Dwellings') & (data['Value'].notna())]
dwellings_group = dwellings_slice.groupby(['AreaCode'])
dwellings_latest_years = dwellings_group.Year.max()
dwellings_names = dwellings_group.AreaName.last().str.strip()

In [3]:
vacants_slice = data.loc[(data['Measure']=='AllVacants') & (data['Value'].notna())]
vacants_group = vacants_slice.groupby(['AreaCode'])
vacants_latest_years = vacants_group.Year.max()
codes_not_in_dwellings = []
for code in vacants_latest_years.index.unique():
    if code not in dwellings_latest_years.index.unique():
        codes_not_in_dwellings.append(code)
vacants_latest_years.drop(index=codes_not_in_dwellings, inplace=True)

In [4]:
d = pd.DataFrame(data={'dwellingsLastPublished': dwellings_latest_years, 'vacantsLastPublished': vacants_latest_years,'active': dwellings_latest_years == latest_year, 'name': dwellings_names})

In [5]:
d.dropna()

,dwellingsLastPublished,vacantsLastPublished,active,name
AreaCode,,,,
E06000001,2023,2023.0,True,Hartlepool UA
E06000002,2023,2023.0,True,Middlesbrough UA
E06000003,2023,2023.0,True,Redcar and Cleveland UA
E06000004,2023,2023.0,True,Stockton-on-Tees UA
E06000005,2023,2023.0,True,Darlington UA
...,...,...,...,...
E12000006,2023,2023.0,True,East of England
E12000007,2023,2023.0,True,London
E12000008,2023,2023.0,True,South East


Write this out to a JSON file to power site generation.

In [6]:
d.to_json('src/visualisations/vacant-homes/_data/areas.json',orient='index', indent=4)